In [4]:
import os
import sys
from importlib import reload
from pathlib import Path

import numpy as np
from torch.nn import ReLU, Linear
from torch_geometric.nn import GCNConv

In [5]:
# NOTE: Boilerplate setup for Jupyter imports
sys.path.append(Path(os.getcwd()).parent.as_posix())

import configs as config_module
import enhancer as enhancer_module
import encoders as encoders_module
import scheme.network as network_module
import scheme.data as data_module

reload(config_module)
reload(enhancer_module)
reload(encoders_module)
reload(network_module)
reload(data_module)

PathConfig, TrainConfig = config_module.PathConfig, config_module.TrainConfig
Enhancer = enhancer_module.Enhancer
get_default_encoders = encoders_module.get_default_encoders
GNNConfig, EnhancerData = network_module.NetworkConfig, data_module.EnhancerData

In [6]:
path_config = PathConfig(
    data_dir="../../data"
)

with open(path_config.target_data, "rb") as f:
    unpacked = np.load(f)

    # NOTE: Target dimensions
    data = EnhancerData(
        unpacked["data"],
        unpacked["target"].reshape(-1),
        unpacked["spatial"],
    )

In [7]:
gnn_setup = GNNConfig(
    encoder=[
        GCNConv(data.features.shape[1], 256),
        GCNConv(256, 256),
    ],
    estimator=[
        Linear(256, 128),
        ReLU(),
        Linear(128, 128),
        ReLU(),
        Linear(128, len(np.unique(data.target))),
    ]
)

In [9]:
encoders = [
    encoders_module.DistEncoder(
        max_dist=5,
        cache_dir=path_config.edge_cache,
        note="cora_dist",
    ),
    encoders_module.ReprEncoder(
        neighbor_rate=0.7,
        cache_dir=path_config.edge_cache,
        note="cora_repr",
    ),
]

result = Enhancer.compare_builders(data, gnn_setup, encoders)

cora_dist test: Loss = 1.0856e+00 | Accuracy = 0.7195571955719557
cora_repr test: Loss = 1.4991e+00 | Accuracy = 0.41697416974169743
DistEncoder test: Loss = 1.0283e+00 | Accuracy = 0.6715867158671587
ReprEncoder test: Loss = 1.4701e+00 | Accuracy = 0.4354243542435424


In [10]:
print(result)

Option         accuracy_score    f1_score
-----------  ----------------  ----------
cora_dist            0.719557    0.713762
cora_repr            0.416974    0.398919
DistEncoder          0.671587    0.687473
ReprEncoder          0.435424    0.429701


In [11]:
enhancer = Enhancer(
    net_config=gnn_setup,
    edge_builder=encoders_module.DistEncoder(
        max_dist=5,
        cache_dir=path_config.edge_cache,
        note="cora_dist [max dist = 5]",
    ),
)

In [12]:
gnn = enhancer.fit(data)

In [13]:
enhancer.transform(data)

tensor([[ 0.1206, -0.4277,  0.2256,  ...,  0.4005, -0.4371,  0.3329],
        [ 0.3792, -0.2805,  0.1656,  ...,  0.1598, -0.9276,  0.6825],
        [ 0.5515,  0.2715, -0.2174,  ...,  0.0725, -1.0501,  0.7025],
        ...,
        [ 0.4890, -0.5761, -0.2172,  ...,  0.3201, -1.1752,  1.1274],
        [ 0.2367, -0.3104,  0.0287,  ...,  0.1513, -0.8360,  0.6601],
        [ 0.2236, -0.5275,  0.2437,  ...,  0.2015, -0.7797,  0.5615]],
       grad_fn=<AddBackward0>)